In this notebook, we will showcase how to use OpenOOD's unified evaluator to easily evaluate OOD detection performance w.r.t. a certain ID dataset (e.g., CIFAR-10, ImageNet-1K), given a trained base classifier (e.g., ResNet) and a postprocessor (e.g., MSP, ReAct). Here we use CIFAR-10 as an example (due to speed concern), yet this tutorial also generalizes to larger datasets including ImageNet-1K. **Remember to use GPU in Edit / Notebook settings.**

1. Install OpenOOD with pip and make necessary preparation

In [ ]:
!pip install git+https://github.com/Jingkang50/OpenOOD

  Cloning https://github.com/zjysteven/OpenOOD to /tmp/pip-req-build-l63u9xuh
  Running command git clone --filter=blob:none --quiet https://github.com/zjysteven/OpenOOD /tmp/pip-req-build-l63u9xuh
  Resolved https://github.com/zjysteven/OpenOOD to commit 6e0ade72ea2d959e0e5c4e5b5c9ef0cba60d4ab2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 47.6 MB/s eta 0:00:00
  Created wheel for openood: filename=openood-1.5-py3-none-any.whl size=248245 sha256=2549a084e93cf2c73cf318369e03cc1273d784259116154239bd0811f79211b2
  Stored in directory: /tm

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

In [2]:
import sys
sys.path.insert(0, "/mnt/data/tq2067/OpenOOD") 

In [3]:
# necessary imports
import torch

from openood.evaluation_api import Evaluator
from openood.networks import ResNet18_32x32 # just a wrapper around the ResNet
from openood.networks import ResNet18_224x224 # imagenet200

In [4]:
# download our pre-trained CIFAR-10 classifier
# cifar10
# !gdown 1byGeYxM_PlLjT72wZsMQvP6popJeWBgt
# !unzip cifar10_res18_v1.5.zip

#cifar100
# !gdown 1s-1oNrRtmA0pGefxXJOUVRYpaoAML0C-
# !unzip cifar100_res18_v1.5.zip

# ImageNet-200 (ResNet-18, OpenOOD v1.5)
# !gdown 1ddVmwc8zmzSjdLUO84EuV4Gz1c7vhIAs
# !unzip imagenet200_res18_v1.5.zip
# !y

In [5]:
# load the model
# net = ResNet18_32x32(num_classes=10)
net = ResNet18_224x224(num_classes=200)
net.load_state_dict(
    torch.load('./imagenet200_resnet18_224x224_base_e90_lr0.1_default/s0/best.ckpt')
)
net.cuda()
net.eval();

In [6]:
import os, torch, sys, platform
print("torch:", torch.__version__, "| torch.version.cuda:", torch.version.cuda)
print("torch.cuda.is_available():", torch.cuda.is_available())
print("device_count:", torch.cuda.device_count())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

torch: 2.6.0+cu124 | torch.version.cuda: 12.4
torch.cuda.is_available(): True
device_count: 1
CUDA_VISIBLE_DEVICES: 0


In [7]:
# # load own net for bootood
# net = ResNet18_32x32(num_classes=100) # cifar10/cifar100
# net = ResNet18_224x224(num_classes=200) # imagenet
# net.load_state_dict(torch.load('nets-2509/FNLoss_imagenet200_0.7M_0.2OR_400P1_200P2_mixup.pth'))

# net.cuda()
# net.eval()

In [8]:
# feature extraction for bootood
from openood.evaluation_api.featurebank import feature_hook
from openood.evaluation_api.featurebank import FeatureBank

handle = net.layer4.register_forward_hook(feature_hook)

In [9]:
#@title choose an implemented postprocessor
postprocessor_name = "neco" #@param ["openmax", "msp", "temp_scaling", "odin", "mds", "mds_ensemble", "rmds", "gram", "ebo", "gradnorm", "react", "mls", "klm", "vim", "knn", "dice", "rankfeat", "ash", "she"] {allow-input: true}


2. Initialize an evaluator instance

In [10]:
# Notes:
# 1) The evaluator will automatically download the required datasets given the
# ID dataset specified by `id_name`

# 2) Passing the `postprocessor_name` will use an implemented postprocessor. To
# use your own postprocessor, just make sure that it inherits the BasePostprocessor
# class (see openood/postprocessors/base_postprocessor.py) and pass it to the
# `postprocessor` argument.

# 3) `config_root` points to the directory with OpenOOD's configurations for the
# postprocessors. By default the evaluator will look for the configs that come
# with the OpenOOD module. If you want to use custom configs, clone the repo locally
# and make modifications to OpenOOD/configs.

# 4) As you will see when executing this cell, during the initialization the evaluator
# will automatically run hyperparameter search on ID/OOD validation data (if applicable).
# If you want to use a postprocessor with specific hyperparams, you need
# to clone the OpenOOD repo (or just download the configs folder in the repo).
# Then a) specify the hyperparams and b) set APS_mode to False in the respective postprocessor
# config.

evaluator = Evaluator(
    net,
    id_name='imagenet200',                     # the target ID dataset
    data_root='./data',                    # change if necessary
    config_root=None,                      # see notes above
    preprocessor=None,                     # default preprocessing for the target ID dataset
    postprocessor_name=postprocessor_name, # the postprocessor to use
    postprocessor=None,                    # if you want to use your own postprocessor
    batch_size=200,                        # for certain methods the results can be slightly affected by batch size
    shuffle=False,
    num_workers=2)                         # could use more num_workers outside colab

[neco] IncrementalPCA: ipca_batch=8192, max_fit_samples=50000
[neco] D=25088, n_components=200
[neco] Fitted 200/50000 samples in 12.2s
[neco] Fitted 10000/50000 samples in 903.6s
[neco] Fitted 10200/50000 samples in 930.2s
[neco] Fitted 20000/50000 samples in 1987.0s
[neco] Fitted 20200/50000 samples in 2014.0s
[neco] Fitted 30000/50000 samples in 3071.9s
[neco] Fitted 30200/50000 samples in 3098.0s
[neco] Fitted 40000/50000 samples in 4155.9s
[neco] Fitted 40200/50000 samples in 4182.1s
[neco] Fitted 50000/50000 samples in 5234.7s
[neco] IPCA fit done. total_used=50000, elapsed=5237.5s


In [11]:
# let's do standard OOD detection
# full-spectrum detection is also available with
# `fsood` being True if you are evaluating on ImageNet

# the returned metrics is a dataframe which includes
# AUROC, AUPR, FPR@95 etc.
metrics = evaluator.eval_ood(fsood=False)

Performing inference on imagenet200 test set...


100%|███████████████████████████████████████████| 45/45 [01:13<00:00,  1.63s/it]

Processing near ood...
Performing inference on ssb_hard dataset...



100%|█████████████████████████████████████████| 245/245 [04:17<00:00,  1.05s/it]


Computing metrics on ssb_hard dataset...
FPR@95: 87.31, AUROC: 55.08 AUPR_IN: 22.32, AUPR_OUT: 85.39
──────────────────────────────────────────────────────────────────────

Performing inference on ninco dataset...


...

Computing metrics on textures dataset...
FPR@95: 76.84, AUROC: 64.05 AUPR_IN: 78.14, AUPR_OUT: 45.47
──────────────────────────────────────────────────────────────────────

Performing inference on openimage_o dataset...



100%|███████████████████████████████████████████| 80/80 [01:22<00:00,  1.03s/it]

Computing metrics on openimage_o dataset...
FPR@95: 83.31, AUROC: 59.29 AUPR_IN: 50.70, AUPR_OUT: 68.60
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 81.38, AUROC: 57.68 AUPR_IN: 61.53, AUPR_OUT: 54.40
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████████████████████████| 45/45 [00:45<00:00,  1.00s/it]

             FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
ssb_hard      87.31  55.08    22.32     85.39 86.39
ninco         86.64  56.70    69.29     42.44 86.39
nearood       86.98  55.89    45.81     63.92 86.39
inaturalist   83.98  49.71    55.74     49.12 86.39
textures      76.84  64.05    78.14     45.47 86.39
openimage_o   83.31  59.29    50.70     68.60 86.39
farood        81.38  57.68    61.53     54.40 86.39


4. What you can get from this evaluator

In [ ]:
# there is some useful information stored as attributes
# of the evaluator instance

# evaluator.metrics stores all the evaluation results
# evaluator.scores stores OOD scores and ID predictions

# for more details please see OpenOOD/openood/evaluation_api/evaluator.py

print('Components within evaluator.metrics:\t', evaluator.metrics.keys())
print('Components within evaluator.scores:\t', evaluator.scores.keys())
print('')
print('The predicted ID class of the first 5 samples of CIFAR-100:\t', evaluator.scores['ood']['near']['cifar100'][0][:5])
print('The OOD score of the first 5 samples of CIFAR-100:\t', evaluator.scores['ood']['near']['cifar100'][1][:5])

Components within evaluator.metrics:	 dict_keys(['id_acc', 'csid_acc', 'ood', 'fsood'])
Components within evaluator.scores:	 dict_keys(['id', 'csid', 'ood', 'id_preds', 'id_labels', 'csid_preds', 'csid_labels'])

The predicted ID class of the first 5 samples of CIFAR-100:	 [9 9 9 9 9]
The OOD score of the first 5 samples of CIFAR-100:	 [5.153 5.214 6.402 6.655 5.155]


5. Extending OpenOOD for your own research/development

We try to make OpenOOD extensible and convenient for everyone.


You can evaluate your own trained model as long as it has necessary functions/methods that help it work with the postprocessors (see OpenOOD/openood/resnet18_32x32.py for example).


You can also design your own postprocessor by inheriting the base class (OpenOOD/openood/postprocessors/base_postprocessor.py), and the resulting method can be readily evaluated with OpenOOD.


Feel free to reach out to us if you have furthur suggestions on making OpenOOD more general and easy-to-use!

# 一次循环直接测7个postprocessor

In [4]:
import pandas as pd

# load own net for bootood
net = ResNet18_32x32(num_classes=100)
# net = ResNet18_224x224(num_classes=200) # imagenet
net.load_state_dict(torch.load('nets-2509/job1098293.pth'))
net.cuda()
net.eval()

post_names = ["msp", "entropy","norm", "cos", "euc", "neco", "mspcos"]
results = []

for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar100",
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

all_metrics = pd.concat(results, ignore_index=True)
print(all_metrics)


== Running msp ==
Performing inference on cifar100 test set...


100%|██████████| 45/45 [00:07<00:00,  5.88it/s]

Processing near ood...
Performing inference on cifar10 dataset...



100%|██████████| 50/50 [00:06<00:00,  7.52it/s]

Computing metrics on cifar10 dataset...
FPR@95: 58.14, AUROC: 79.80 AUPR_IN: 80.75, AUPR_OUT: 77.31
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 55.89, AUROC: 79.99 AUPR_IN: 87.23, AUPR_OUT: 66.93
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 169/169 [00:31<00:00,  5.42it/s]

Computing metrics on places365 dataset...
FPR@95: 56.06, AUROC: 80.25 AUPR_IN: 61.48, AUPR_OUT: 91.98
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 53.51, AUROC: 81.17 AUPR_IN: 68.95, AUPR_OUT: 86.60
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:04<00:00, 10.03it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar10     58.13  79.74    80.69     77.10 78.39
tin         48.48  84.08    88.98     74.93 78.39
nearood     53.31  81.91    84.84     76.01 78.39
mnist       48.37  81.74    56.73     96.08 78.39
svhn        53.72  82.69    70.36     91.40 78.39
texture     55.89  79.99    87.23     66.93 78.39
places365   56.06  80.25    61.48     91.98 78.39
farood      53.51  81.17    68.95     86.60 78.39
       FPR@95      AUROC    AUPR_IN   AUPR_OUT        ACC postprocessor
0   58.144444  79.796747  80.746989  77.308764  78.388889           msp
1   48.433333  84.108442  88.958887  75.013796  78.388889           msp
2   53.288889  81.952594  84.852938  76.161280  78.388889           msp
3   48.277778  81.811014  57.082824  96.122563  78.388889           msp
4   53.588889  82.751560  70.597258  91.466233  78.388889           msp
5   55.711111  80.092728  87.383134  67.272928  78.388889           msp
6   55.933333  80.336821  61.955275  92.071032

# for feature separation 对应的修改过后的resnet

In [9]:
# ===== PreAct CIFAR-ResNet (3 stages, exact key match) + strict load + OpenOOD eval =====
import re
import torch
import torch.nn as nn
import pandas as pd
from openood.evaluation_api import Evaluator

# ---------- 1) 读取并清洗 state_dict ----------
def load_clean_state_dict(ckpt_path):
    raw = torch.load(ckpt_path, map_location='cpu')
    sd = raw['state_dict'] if isinstance(raw, dict) and 'state_dict' in raw else raw
    # 去掉 DataParallel 前缀
    sd = {k.replace('module.', ''): v for k, v in sd.items()}
    return sd

# ---------- 2) 从 ckpt 推断结构 ----------
def infer_cifar3_from_ckpt(sd):
    # stem：优先 conv1.weight
    if 'conv1.weight' in sd:
        stem = sd['conv1.weight'].shape[0]
    else:
        stem = 16  # 兜底

    # block 数：看 block{1,2,3}.layer.<idx>.conv1.weight 最大 idx
    def count_blocks(bid):
        patt = re.compile(rf'^block{bid}\.layer\.(\d+)\.conv1\.weight$')
        max_i = -1
        for k in sd:
            m = patt.match(k)
            if m:
                max_i = max(max_i, int(m.group(1)))
        return max_i + 1 if max_i >= 0 else 0

    n1, n2, n3 = count_blocks(1), count_blocks(2), count_blocks(3)
    if n1*n2*n3 == 0:
        raise RuntimeError("找不到 block{1,2,3}.layer.<i>.conv1.weight 这样的键，命名可能不同。")

    # 每个 stage 的 out_channels：取该 stage 第一块 conv1 的输出通道
    def stage_out(bid):
        k = f'block{bid}.layer.0.conv1.weight'
        if k in sd:
            return sd[k].shape[0]
        # 兜底：任意 i
        patt = re.compile(rf'^block{bid}\.layer\.\d+\.conv1\.weight$')
        for kk in sd:
            if patt.match(kk):
                return sd[kk].shape[0]
        return None

    c1, c2, c3 = stage_out(1), stage_out(2), stage_out(3)
    # 类别数
    if 'fc.weight' in sd:
        num_classes = sd['fc.weight'].shape[0]
    else:
        raise RuntimeError("缺少 fc.weight，无法推断 num_classes。")

    # 兜底通道
    if c1 is None: c1 = 32
    if c2 is None: c2 = max(64, c1*2)
    if c3 is None: c3 = max(128, c2*2)

    return stem, (n1, n2, n3), (c1, c2, c3), num_classes

# ---------- 3) Pre-Activation BasicBlock（bn1 在 conv1 之前） ----------
class PreActBasicBlockCIFAR3(nn.Module):
    """
    键名与 ckpt 对齐：
      - bn1 (通道 = in_planes), conv1 (in->out)
      - bn2 (通道 = out_planes), conv2 (out->out)
      - convShortcut 仅在降采样/调通道时出现（无 BN）
    """
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.bn1   = nn.BatchNorm2d(in_planes)
        self.relu  = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn2   = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)

        self.convShortcut = None
        if stride != 1 or in_planes != planes:
            self.convShortcut = nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(x))
        if self.convShortcut is not None:
            identity = self.convShortcut(out)  # preact 变体中常把激活后的输入走 shortcut
        out = self.conv1(out)
        out = self.conv2(self.relu(self.bn2(out)))
        out = out + identity
        return out

# ---------- 4) 3-stage PreAct CIFAR-ResNet，键名对齐 ----------
class PreActCIFARResNet3(nn.Module):
    """
    stem: conv1(3->stem), ReLU（无 stem BN，避免与 ckpt 顶层 bn1 冲突）
    block1.layer: n1 blocks, out c1 （第一个 block stride=1）
    block2.layer: n2 blocks, out c2 （第一个 block stride=2）
    block3.layer: n3 blocks, out c3 （第一个 block stride=2）
    fc: Linear(c3 -> num_classes)
    """
    def __init__(self, stem_channels, n_blocks, stage_channels, num_classes):
        super().__init__()
        n1, n2, n3 = n_blocks
        c1, c2, c3 = stage_channels

        self.conv1 = nn.Conv2d(3, stem_channels, 3, 1, 1, bias=False)
        self.relu  = nn.ReLU(inplace=True)

        def make_stage(in_planes, out_planes, blocks, downsample):
            layers = []
            for i in range(blocks):
                stride = (2 if (i == 0 and downsample) else 1)
                layers.append(PreActBasicBlockCIFAR3(in_planes, out_planes, stride))
                in_planes = out_planes
            return nn.Sequential(*layers), in_planes

        in_planes = stem_channels
        self.block1 = nn.Module()
        self.block1.layer, in_planes = make_stage(in_planes, c1, n1, downsample=False)

        self.block2 = nn.Module()
        self.block2.layer, in_planes = make_stage(in_planes, c2, n2, downsample=True)

        self.block3 = nn.Module()
        self.block3.layer, in_planes = make_stage(in_planes, c3, n3, downsample=True)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(c3, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.block1.layer(x)
        x = self.block2.layer(x)
        x = self.block3.layer(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# ---------- 5) 构建模型 + 过滤 ckpt 键 + 严格加载 ----------
def build_and_load(ckpt_path):
    sd = load_clean_state_dict(ckpt_path)
    stem, n_blocks, stage_channels, num_classes = infer_cifar3_from_ckpt(sd)
    print(f'[inferred] stem={stem}, n_blocks={n_blocks}, stage_channels={stage_channels}, num_classes={num_classes}')

    net = PreActCIFARResNet3(
        stem_channels=stem,
        n_blocks=n_blocks,
        stage_channels=stage_channels,
        num_classes=num_classes
    )

    model_sd = net.state_dict()
    # 只保留模型里存在、且形状匹配的键；剔除顶层 bn1.* 等多余项
    filtered = {}
    mism = []
    for k, v in sd.items():
        if k in model_sd:
            if model_sd[k].shape == v.shape:
                filtered[k] = v
            else:
                mism.append((k, tuple(v.shape), tuple(model_sd[k].shape)))
    if mism:
        print(">> 仍有形状不一致（前10条）：")
        for i,(k,a,b) in enumerate(mism[:10]):
            print(f'   {i+1}. {k}: ckpt {a} vs model {b}')
        # 若出现，通常是结构还没完全对齐，贴出这些键我来再改一版

    # 检查是否还有模型键没被覆盖（正常情况下=0）
    missing_after_filter = [k for k in model_sd.keys() if k not in filtered]
    if missing_after_filter:
        print(">> 模型中未被覆盖的键数量：", len(missing_after_filter))
        for k in missing_after_filter[:10]:
            print("   -", k)

    # 严格加载“过滤后的”权重
    net.load_state_dict(filtered, strict=True)
    return net

# ---------- 6) 一键跑评测 ----------
ckpt_path = 'nets-2509/cifar10_ours.pth'  # ← 换成你的路径
net = build_and_load(ckpt_path)
net.cuda().eval()
print('Loaded OK (PreAct, 3-stage, exact-key match).')

post_names = ["mspcos"] #"msp", "entropy", "norm", "cos", "euc", "neco"
results = []
for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar10",
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

all_metrics = pd.concat(results, ignore_index=True)
print(all_metrics)
# ============================ END ============================


[inferred] stem=16, n_blocks=(6, 6, 6), stage_channels=(32, 64, 128), num_classes=10
Loaded OK (PreAct, 3-stage, exact-key match).
== Running mspcos ==
Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:04<00:00, 10.35it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:04<00:00, 10.03it/s]

Computing metrics on cifar100 dataset...
FPR@95: 55.56, AUROC: 87.30 AUPR_IN: 86.46, AUPR_OUT: 81.05
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 10.53, AUROC: 92.84 AUPR_IN: 96.73, AUPR_OUT: 76.98
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [00:32<00:00,  5.33it/s]

Computing metrics on places365 dataset...
FPR@95: 10.97, AUROC: 92.76 AUPR_IN: 87.31, AUPR_OUT: 95.52
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 12.21, AUROC: 92.96 AUPR_IN: 88.59, AUPR_OUT: 91.14
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:03<00:00, 11.30it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    55.56  87.30    86.46     81.05 86.06
tin         19.20  90.91    92.88     80.98 86.06
nearood     37.38  89.10    89.67     81.01 86.06
mnist       17.44  92.62    76.55     98.06 86.06
svhn         9.90  93.63    93.79     93.99 86.06
texture     10.53  92.84    96.73     76.98 86.06
places365   10.97  92.76    87.31     95.52 86.06
farood      12.21  92.96    88.59     91.14 86.06
      FPR@95      AUROC    AUPR_IN   AUPR_OUT        ACC postprocessor
0  55.555556  87.296423  86.462443  81.047119  86.055556        mspcos
1  19.200000  90.908106  92.881937  80.982356  86.055556        mspcos
2  37.377778  89.102265  89.672190  81.014738  86.055556        mspcos
3  17.444444  92.621362  76.545544  98.064128  86.055556        mspcos
4   9.900000  93.628603  93.793895  93.990015  86.055556        mspcos
5  10.533333  92.841593  96.727207  76.978055  86.055556        mspcos
6  10.966667  92.763989  87.308693  95.523803  86.055

In [10]:
# ===== PreAct CIFAR-ResNet (3 stages) + strict load from CKPT + OpenOOD eval (CIFAR-100) =====
import re
import torch
import torch.nn as nn
import pandas as pd
from openood.evaluation_api import Evaluator

# ---------- 1) 读取并清洗 state_dict ----------
def load_clean_state_dict(ckpt_path):
    raw = torch.load(ckpt_path, map_location='cpu')
    sd = raw['state_dict'] if isinstance(raw, dict) and 'state_dict' in raw else raw
    # 去掉 DataParallel 前缀
    sd = {k.replace('module.', ''): v for k, v in sd.items()}
    return sd

# ---------- 2) 从 ckpt 推断结构 ----------
def infer_cifar3_from_ckpt(sd):
    # stem：优先 conv1.weight
    stem = sd['conv1.weight'].shape[0] if 'conv1.weight' in sd else 16

    # block 数：block{1,2,3}.layer.<idx>.conv1.weight 最大 idx
    def count_blocks(bid):
        patt = re.compile(rf'^block{bid}\.layer\.(\d+)\.conv1\.weight$')
        max_i = -1
        for k in sd:
            m = patt.match(k)
            if m: max_i = max(max_i, int(m.group(1)))
        return max_i + 1 if max_i >= 0 else 0

    n1, n2, n3 = count_blocks(1), count_blocks(2), count_blocks(3)
    if n1*n2*n3 == 0:
        raise RuntimeError("找不到 block{1,2,3}.layer.<i>.conv1.weight 这样的键，命名可能不同。")

    # 每个 stage 的 out_channels：该 stage 第一块 conv1 的输出通道
    def stage_out(bid):
        k0 = f'block{bid}.layer.0.conv1.weight'
        if k0 in sd: return sd[k0].shape[0]
        patt = re.compile(rf'^block{bid}\.layer\.\d+\.conv1\.weight$')
        for kk in sd:
            if patt.match(kk): return sd[kk].shape[0]
        return None

    c1, c2, c3 = stage_out(1), stage_out(2), stage_out(3)

    # 类别数
    if 'fc.weight' in sd:
        num_classes = sd['fc.weight'].shape[0]
    else:
        raise RuntimeError("缺少 fc.weight，无法推断 num_classes。")

    # 兜底通道
    if c1 is None: c1 = 32
    if c2 is None: c2 = max(64, c1*2)
    if c3 is None: c3 = max(128, c2*2)

    return stem, (n1, n2, n3), (c1, c2, c3), num_classes

# ---------- 3) Pre-Activation BasicBlock（bn1 在 conv1 之前） ----------
class PreActBasicBlockCIFAR3(nn.Module):
    """
    键名与 ckpt 对齐：
      - bn1 (通道 = in_planes), conv1 (in->out)
      - bn2 (通道 = out_planes), conv2 (out->out)
      - convShortcut 仅在降采样/调通道时出现（无 BN）
    """
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.bn1   = nn.BatchNorm2d(in_planes)
        self.relu  = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn2   = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)

        self.convShortcut = None
        if stride != 1 or in_planes != planes:
            self.convShortcut = nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(x))
        if self.convShortcut is not None:
            identity = self.convShortcut(out)  # preact风格：激活后的分支走shortcut
        out = self.conv1(out)
        out = self.conv2(self.relu(self.bn2(out)))
        out = out + identity
        return out

# ---------- 4) 3-stage PreAct CIFAR-ResNet，键名对齐 ----------
class PreActCIFARResNet3(nn.Module):
    """
    stem: conv1(3->stem), ReLU（无 stem BN）
    block1.layer: n1 blocks, out c1 （第一个 block stride=1）
    block2.layer: n2 blocks, out c2 （第一个 block stride=2）
    block3.layer: n3 blocks, out c3 （第一个 block stride=2）
    fc: Linear(c3 -> num_classes)
    """
    def __init__(self, stem_channels, n_blocks, stage_channels, num_classes):
        super().__init__()
        n1, n2, n3 = n_blocks
        c1, c2, c3 = stage_channels

        self.conv1 = nn.Conv2d(3, stem_channels, 3, 1, 1, bias=False)
        self.relu  = nn.ReLU(inplace=True)

        def make_stage(in_planes, out_planes, blocks, downsample):
            layers = []
            for i in range(blocks):
                stride = (2 if (i == 0 and downsample) else 1)
                layers.append(PreActBasicBlockCIFAR3(in_planes, out_planes, stride))
                in_planes = out_planes
            return nn.Sequential(*layers), in_planes

        in_planes = stem_channels
        self.block1 = nn.Module()
        self.block1.layer, in_planes = make_stage(in_planes, c1, n1, downsample=False)

        self.block2 = nn.Module()
        self.block2.layer, in_planes = make_stage(in_planes, c2, n2, downsample=True)

        self.block3 = nn.Module()
        self.block3.layer, in_planes = make_stage(in_planes, c3, n3, downsample=True)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(c3, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.block1.layer(x)
        x = self.block2.layer(x)
        x = self.block3.layer(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# ---------- 5) 构建模型 + 过滤 ckpt 键 + 严格加载 ----------
def build_and_load(ckpt_path, expect_classes=100):
    sd = load_clean_state_dict(ckpt_path)
    stem, n_blocks, stage_channels, num_classes = infer_cifar3_from_ckpt(sd)
    print(f'[inferred] stem={stem}, n_blocks={n_blocks}, stage_channels={stage_channels}, num_classes={num_classes}')
    if num_classes != expect_classes:
        print(f'>>> Warning: ckpt num_classes={num_classes}, but you are evaluating CIFAR-{expect_classes}. '
              f'确保权重确实是 CIFAR-{expect_classes} 的。')

    net = PreActCIFARResNet3(
        stem_channels=stem,
        n_blocks=n_blocks,
        stage_channels=stage_channels,
        num_classes=num_classes  # 用 ckpt 里的类别数，通常=100
    )

    model_sd = net.state_dict()
    # 只保留模型里存在、且形状匹配的键；（顶层 bn1.* 若存在且不匹配会被自然过滤）
    filtered = {}
    mism = []
    for k, v in sd.items():
        if k in model_sd:
            if model_sd[k].shape == v.shape:
                filtered[k] = v
            else:
                mism.append((k, tuple(v.shape), tuple(model_sd[k].shape)))
    if mism:
        print(">> 仍有形状不一致（前10条）：")
        for i,(k,a,b) in enumerate(mism[:10]):
            print(f'   {i+1}. {k}: ckpt {a} vs model {b}')

    # 检查是否还有模型键没被覆盖（正常情况下尽量少）
    missing_after_filter = [k for k in model_sd.keys() if k not in filtered]
    if missing_after_filter:
        print(">> 模型中未被覆盖的键数量：", len(missing_after_filter))
        for k in missing_after_filter[:10]:
            print("   -", k)

    net.load_state_dict(filtered, strict=True)
    return net

# ---------- 6) 一键跑评测（CIFAR-100） ----------
ckpt_path = 'nets-2509/cifar100_ours.pth'  # ← 换成你的 CIFAR-100 权重路径
net = build_and_load(ckpt_path, expect_classes=100)
net.cuda().eval()
print('Loaded OK (PreAct, 3-stage, CIFAR-100).')

post_names = ["mspcos"] #"msp", "entropy", "norm", "cos", "euc", "neco"
results = []
for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar100",            # ★ 这里改成 cifar100
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

all_metrics = pd.concat(results, ignore_index=True)
print(all_metrics)
# ============================ END ============================

[inferred] stem=16, n_blocks=(6, 6, 6), stage_channels=(32, 64, 128), num_classes=100
Loaded OK (PreAct, 3-stage, CIFAR-100).
== Running mspcos ==
Performing inference on cifar100 test set...


100%|██████████| 45/45 [00:05<00:00,  8.76it/s]

Processing near ood...
Performing inference on cifar10 dataset...



100%|██████████| 50/50 [00:04<00:00, 11.65it/s]

Computing metrics on cifar10 dataset...
FPR@95: 63.34, AUROC: 72.41 AUPR_IN: 74.31, AUPR_OUT: 68.31
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 56.12, AUROC: 70.65 AUPR_IN: 83.76, AUPR_OUT: 48.85
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 169/169 [00:31<00:00,  5.30it/s]

Computing metrics on places365 dataset...
FPR@95: 58.94, AUROC: 69.95 AUPR_IN: 51.43, AUPR_OUT: 85.37
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 64.80, AUROC: 68.19 AUPR_IN: 53.62, AUPR_OUT: 75.56
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:04<00:00,  9.76it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar10     63.34  72.41    74.31     68.31 54.07
tin         59.84  70.99    79.77     55.93 54.07
nearood     61.59  71.70    77.04     62.12 54.07
mnist       84.28  61.77    20.50     88.98 54.07
svhn        59.87  70.36    58.78     79.05 54.07
texture     56.12  70.65    83.76     48.85 54.07
places365   58.94  69.95    51.43     85.37 54.07
farood      64.80  68.19    53.62     75.56 54.07
      FPR@95      AUROC    AUPR_IN   AUPR_OUT        ACC postprocessor
0  63.344444  72.406326  74.311473  68.307865  54.066667        mspcos
1  59.844444  70.994916  79.772540  55.929507  54.066667        mspcos
2  61.594444  71.700621  77.042007  62.118686  54.066667        mspcos
3  84.277778  61.773675  20.501733  88.981041  54.066667        mspcos
4  59.866667  70.362172  58.781172  79.047324  54.066667        mspcos
5  56.122222  70.653969  83.755866  48.845392  54.066667        mspcos
6  58.944444  69.954940  51.433515  85.371566  54.066

# 1021

In [6]:
import torch
import pandas as pd

# =============================
# 1. 加载网络
# =============================
net = ResNet18_32x32(num_classes=10)
# net = ResNet18_224x224(num_classes=200)  # 若是 ImageNet
net.load_state_dict(torch.load('nets-cifar10/job1126841.pth'))
net.cuda()
net.eval()

# =============================
# 2. 运行多个后处理器
# =============================
post_names = ["msp", "entropy", "norm", "cos", "euc", "neco"]
results = []

for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar10",
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

# =============================
# 3. 汇总与格式化输出
# =============================
if results:
    # === 合并 & 标准化 ===
    all_metrics = pd.concat(results, ignore_index=False)
    if 'dataset' not in all_metrics.columns:
        if all_metrics.index.name:
            all_metrics = all_metrics.reset_index().rename(columns={all_metrics.index.name: 'dataset'})
        else:
            all_metrics = all_metrics.reset_index().rename(columns={'index': 'dataset'})
    # 数值化指标
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in all_metrics.columns:
            all_metrics[c] = pd.to_numeric(all_metrics[c], errors='coerce')

    # 只保留 nearood / farood
    cols = ['dataset', 'postprocessor', 'FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']
    have_acc = 'ACC' in all_metrics.columns
    keep_cols = [c for c in cols if (c != 'ACC' or have_acc)]
    summary = all_metrics[all_metrics['dataset'].isin(['nearood', 'farood'])][keep_cols].copy()
    summary = summary.sort_values(['postprocessor', 'dataset']).reset_index(drop=True)

        # === 打印格式化表格 ===
    def fmt(x):
        return '' if pd.isna(x) else f'{x:.2f}'

    header_cols = ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT'] + (['ACC'] if have_acc else [])
    print('\t\t' + '\t'.join(header_cols))

    # === 按原始 post_names 顺序输出 ===
    for pp in post_names:
        df_pp = summary[summary['postprocessor'] == pp]
        def get_vals(row):
            if row.empty:
                return [''] * len(header_cols)
            vals = [fmt(row.iloc[0].get(k, None)) for k in header_cols]
            return vals

        vals_n = get_vals(df_pp[df_pp['dataset'] == 'nearood'])
        vals_f = get_vals(df_pp[df_pp['dataset'] == 'farood'])

        print(f"{pp}\tnearood\t" + '\t'.join(vals_n))
        print(f"\tfarood\t" + '\t'.join(vals_f))

    # === 计算最佳 AUROC ===
    best_near = summary[summary['dataset'] == 'nearood'].sort_values('AUROC', ascending=False).head(1)
    best_far  = summary[summary['dataset'] == 'farood' ].sort_values('AUROC', ascending=False).head(1)

    best_near_auroc = fmt(best_near.iloc[0]['AUROC']) if not best_near.empty else ''
    best_near_name  = best_near.iloc[0]['postprocessor'] if not best_near.empty else ''
    best_far_auroc  = fmt(best_far.iloc[0]['AUROC']) if not best_far.empty else ''
    best_far_name   = best_far.iloc[0]['postprocessor'] if not best_far.empty else ''

    acc_val = ''
    if have_acc:
        cand = pd.concat([best_near[['ACC']], best_far[['ACC']]]) if not best_near.empty and not best_far.empty else summary[['ACC']]
        acc_num = pd.to_numeric(cand['ACC'], errors='coerce').dropna()
        acc_val = fmt(acc_num.iloc[0]) if len(acc_num) else ''

    print()
    tail_header = ['FPR@95', 'Best NearOOD AUROC', 'postProcessorname',
                   'Best FarOOD AUROC', 'postProcessorname'] + (['ACC'] if have_acc else [])
    print('\t'.join(tail_header))
    tail_row = ['', best_near_auroc, best_near_name, best_far_auroc, best_far_name] + ([acc_val] if have_acc else [])
    print('\t'.join(tail_row))

    # === 保存结果（两位小数） ===
    summary_rounded = summary.copy()
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in summary_rounded.columns:
            summary_rounded[c] = summary_rounded[c].round(2)
    summary_rounded.to_csv("nets-cifar10/job1126841.csv", index=False, float_format="%.2f")

else:
    print("No results produced.")

== Running msp ==
Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:06<00:00,  7.47it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:06<00:00,  7.29it/s]

Computing metrics on cifar100 dataset...
FPR@95: 85.13, AUROC: 82.15 AUPR_IN: 75.02, AUPR_OUT: 82.38
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 65.93, AUROC: 72.98 AUPR_IN: 82.83, AUPR_OUT: 59.72
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [00:58<00:00,  3.03it/s]

Computing metrics on places365 dataset...
FPR@95: 75.16, AUROC: 69.19 AUPR_IN: 42.52, AUPR_OUT: 88.23
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 67.95, AUROC: 68.56 AUPR_IN: 55.49, AUPR_OUT: 79.95
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:05<00:00,  8.27it/s]


           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    70.14  74.15    75.14     71.74 93.70
tin         69.28  72.52    77.25     66.57 93.70
nearood     69.71  73.33    76.19     69.15 93.70
mnist       67.07  66.16    39.70     91.62 93.70
svhn        63.66  65.91    56.89     80.25 93.70
texture     65.93  72.98    82.83     59.72 93.70
places365   75.16  69.19    42.52     88.23 93.70
farood      67.95  68.56    55.49     79.95 93.70
		FPR@95	AUROC	AUPR_IN	AUPR_OUT	ACC
msp	nearood	81.23	83.29	77.68	82.48	93.70
	farood	69.59	86.22	60.59	91.55	93.70
entropy	nearood	82.47	83.30	77.37	83.22	93.70
	farood	71.56	86.19	59.86	91.91	93.70
norm	nearood	42.27	87.33	89.33	84.22	93.70
	farood	29.17	91.80	86.38	93.83	93.70
cos	nearood	70.73	76.55	77.88	69.32	93.70
	farood	54.82	81.49	67.97	85.04	93.70
euc	nearood	70.73	76.55	77.88	69.32	93.70
	farood	54.82	81.49	67.97	85.04	93.70
neco	nearood	69.71	73.33	76.19	69.15	93.70
	farood	67.95	68.56	55.49	79.95	93.70

FPR@95	Best NearOOD AUROC	p

In [7]:
import torch
import pandas as pd

# =============================
# 1. 加载网络
# =============================
net = ResNet18_32x32(num_classes=10)
# net = ResNet18_224x224(num_classes=200)  # 若是 ImageNet
net.load_state_dict(torch.load('nets-cifar10/job1126845.pth'))
net.cuda()
net.eval()

# =============================
# 2. 运行多个后处理器
# =============================
post_names = ["msp", "entropy", "norm", "cos", "euc", "neco"]
results = []

for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar10",
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

# =============================
# 3. 汇总与格式化输出
# =============================
if results:
    # === 合并 & 标准化 ===
    all_metrics = pd.concat(results, ignore_index=False)
    if 'dataset' not in all_metrics.columns:
        if all_metrics.index.name:
            all_metrics = all_metrics.reset_index().rename(columns={all_metrics.index.name: 'dataset'})
        else:
            all_metrics = all_metrics.reset_index().rename(columns={'index': 'dataset'})
    # 数值化指标
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in all_metrics.columns:
            all_metrics[c] = pd.to_numeric(all_metrics[c], errors='coerce')

    # 只保留 nearood / farood
    cols = ['dataset', 'postprocessor', 'FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']
    have_acc = 'ACC' in all_metrics.columns
    keep_cols = [c for c in cols if (c != 'ACC' or have_acc)]
    summary = all_metrics[all_metrics['dataset'].isin(['nearood', 'farood'])][keep_cols].copy()
    summary = summary.sort_values(['postprocessor', 'dataset']).reset_index(drop=True)

        # === 打印格式化表格 ===
    def fmt(x):
        return '' if pd.isna(x) else f'{x:.2f}'

    header_cols = ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT'] + (['ACC'] if have_acc else [])
    print('\t\t' + '\t'.join(header_cols))

    # === 按原始 post_names 顺序输出 ===
    for pp in post_names:
        df_pp = summary[summary['postprocessor'] == pp]
        def get_vals(row):
            if row.empty:
                return [''] * len(header_cols)
            vals = [fmt(row.iloc[0].get(k, None)) for k in header_cols]
            return vals

        vals_n = get_vals(df_pp[df_pp['dataset'] == 'nearood'])
        vals_f = get_vals(df_pp[df_pp['dataset'] == 'farood'])

        print(f"{pp}\tnearood\t" + '\t'.join(vals_n))
        print(f"\tfarood\t" + '\t'.join(vals_f))

    # === 计算最佳 AUROC ===
    best_near = summary[summary['dataset'] == 'nearood'].sort_values('AUROC', ascending=False).head(1)
    best_far  = summary[summary['dataset'] == 'farood' ].sort_values('AUROC', ascending=False).head(1)

    best_near_auroc = fmt(best_near.iloc[0]['AUROC']) if not best_near.empty else ''
    best_near_name  = best_near.iloc[0]['postprocessor'] if not best_near.empty else ''
    best_far_auroc  = fmt(best_far.iloc[0]['AUROC']) if not best_far.empty else ''
    best_far_name   = best_far.iloc[0]['postprocessor'] if not best_far.empty else ''

    acc_val = ''
    if have_acc:
        cand = pd.concat([best_near[['ACC']], best_far[['ACC']]]) if not best_near.empty and not best_far.empty else summary[['ACC']]
        acc_num = pd.to_numeric(cand['ACC'], errors='coerce').dropna()
        acc_val = fmt(acc_num.iloc[0]) if len(acc_num) else ''

    print()
    tail_header = ['FPR@95', 'Best NearOOD AUROC', 'postProcessorname',
                   'Best FarOOD AUROC', 'postProcessorname'] + (['ACC'] if have_acc else [])
    print('\t'.join(tail_header))
    tail_row = ['', best_near_auroc, best_near_name, best_far_auroc, best_far_name] + ([acc_val] if have_acc else [])
    print('\t'.join(tail_row))

    # === 保存结果（两位小数） ===
    summary_rounded = summary.copy()
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in summary_rounded.columns:
            summary_rounded[c] = summary_rounded[c].round(2)
    summary_rounded.to_csv("nets-cifar10/job1126845.csv", index=False, float_format="%.2f")

else:
    print("No results produced.")

== Running msp ==
Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:05<00:00,  8.30it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:05<00:00,  8.15it/s]

Computing metrics on cifar100 dataset...
FPR@95: 85.13, AUROC: 82.15 AUPR_IN: 75.02, AUPR_OUT: 82.38
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 65.93, AUROC: 72.98 AUPR_IN: 82.83, AUPR_OUT: 59.72
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [00:57<00:00,  3.04it/s]

Computing metrics on places365 dataset...
FPR@95: 75.16, AUROC: 69.19 AUPR_IN: 42.52, AUPR_OUT: 88.23
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 67.95, AUROC: 68.56 AUPR_IN: 55.49, AUPR_OUT: 79.95
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:05<00:00,  7.82it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    70.14  74.15    75.14     71.74 93.70
tin         69.28  72.52    77.25     66.57 93.70
nearood     69.71  73.33    76.19     69.15 93.70
mnist       67.07  66.16    39.70     91.62 93.70
svhn        63.66  65.91    56.89     80.25 93.70
texture     65.93  72.98    82.83     59.72 93.70
places365   75.16  69.19    42.52     88.23 93.70
farood      67.95  68.56    55.49     79.95 93.70
		FPR@95	AUROC	AUPR_IN	AUPR_OUT	ACC
msp	nearood	81.23	83.29	77.68	82.48	93.70
	farood	69.59	86.22	60.59	91.55	93.70
entropy	nearood	82.47	83.30	77.37	83.22	93.70
	farood	71.56	86.19	59.86	91.91	93.70
norm	nearood	42.27	87.33	89.33	84.22	93.70
	farood	29.17	91.80	86.38	93.83	93.70
cos	nearood	70.73	76.55	77.88	69.32	93.70
	farood	54.82	81.49	67.97	85.04	93.70
euc	nearood	70.73	76.55	77.88	69.32	93.70
	farood	54.82	81.49	67.97	85.04	93.70
neco	nearood	69.71	73.33	76.19	69.15	93.70
	farood	67.95	68.56	55.49	79.95	93.70

FPR@95	Best NearOOD AUROC	p

In [8]:
import torch
import pandas as pd


# =============================
# 1. 加载网络
# =============================
net = ResNet18_32x32(num_classes=10)
# net = ResNet18_224x224(num_classes=200)  # 若是 ImageNet
net.load_state_dict(torch.load('nets-cifar10/job1126847.pth'))
net.cuda()
net.eval()

# =============================
# 2. 运行多个后处理器
# =============================
post_names = ["msp", "entropy", "norm", "cos", "euc", "neco"]
results = []

for name in post_names:
    print(f"== Running {name} ==")
    evaluator = Evaluator(
        net=net,
        id_name="cifar10",
        data_root="./data",
        config_root=None,
        preprocessor=None,
        postprocessor_name=name,
        postprocessor=None,
        batch_size=200,
        shuffle=False,
        num_workers=2
    )
    df = evaluator.eval_ood(fsood=False).copy()
    df["postprocessor"] = name
    results.append(df)

# =============================
# 3. 汇总与格式化输出
# =============================
if results:
    # === 合并 & 标准化 ===
    all_metrics = pd.concat(results, ignore_index=False)
    if 'dataset' not in all_metrics.columns:
        if all_metrics.index.name:
            all_metrics = all_metrics.reset_index().rename(columns={all_metrics.index.name: 'dataset'})
        else:
            all_metrics = all_metrics.reset_index().rename(columns={'index': 'dataset'})
    # 数值化指标
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in all_metrics.columns:
            all_metrics[c] = pd.to_numeric(all_metrics[c], errors='coerce')

    # 只保留 nearood / farood
    cols = ['dataset', 'postprocessor', 'FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']
    have_acc = 'ACC' in all_metrics.columns
    keep_cols = [c for c in cols if (c != 'ACC' or have_acc)]
    summary = all_metrics[all_metrics['dataset'].isin(['nearood', 'farood'])][keep_cols].copy()
    summary = summary.sort_values(['postprocessor', 'dataset']).reset_index(drop=True)

        # === 打印格式化表格 ===
    def fmt(x):
        return '' if pd.isna(x) else f'{x:.2f}'

    header_cols = ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT'] + (['ACC'] if have_acc else [])
    print('\t\t' + '\t'.join(header_cols))

    # === 按原始 post_names 顺序输出 ===
    for pp in post_names:
        df_pp = summary[summary['postprocessor'] == pp]
        def get_vals(row):
            if row.empty:
                return [''] * len(header_cols)
            vals = [fmt(row.iloc[0].get(k, None)) for k in header_cols]
            return vals

        vals_n = get_vals(df_pp[df_pp['dataset'] == 'nearood'])
        vals_f = get_vals(df_pp[df_pp['dataset'] == 'farood'])

        print(f"{pp}\tnearood\t" + '\t'.join(vals_n))
        print(f"\tfarood\t" + '\t'.join(vals_f))

    # === 计算最佳 AUROC ===
    best_near = summary[summary['dataset'] == 'nearood'].sort_values('AUROC', ascending=False).head(1)
    best_far  = summary[summary['dataset'] == 'farood' ].sort_values('AUROC', ascending=False).head(1)

    best_near_auroc = fmt(best_near.iloc[0]['AUROC']) if not best_near.empty else ''
    best_near_name  = best_near.iloc[0]['postprocessor'] if not best_near.empty else ''
    best_far_auroc  = fmt(best_far.iloc[0]['AUROC']) if not best_far.empty else ''
    best_far_name   = best_far.iloc[0]['postprocessor'] if not best_far.empty else ''

    acc_val = ''
    if have_acc:
        cand = pd.concat([best_near[['ACC']], best_far[['ACC']]]) if not best_near.empty and not best_far.empty else summary[['ACC']]
        acc_num = pd.to_numeric(cand['ACC'], errors='coerce').dropna()
        acc_val = fmt(acc_num.iloc[0]) if len(acc_num) else ''

    print()
    tail_header = ['FPR@95', 'Best NearOOD AUROC', 'postProcessorname',
                   'Best FarOOD AUROC', 'postProcessorname'] + (['ACC'] if have_acc else [])
    print('\t'.join(tail_header))
    tail_row = ['', best_near_auroc, best_near_name, best_far_auroc, best_far_name] + ([acc_val] if have_acc else [])
    print('\t'.join(tail_row))

    # === 保存结果（两位小数） ===
    summary_rounded = summary.copy()
    for c in ['FPR@95', 'AUROC', 'AUPR_IN', 'AUPR_OUT', 'ACC']:
        if c in summary_rounded.columns:
            summary_rounded[c] = summary_rounded[c].round(2)
    summary_rounded.to_csv("nets-cifar10/job1126847.csv", index=False, float_format="%.2f")

else:
    print("No results produced.")

== Running msp ==
Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:06<00:00,  7.40it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:06<00:00,  7.13it/s]

Computing metrics on cifar100 dataset...
FPR@95: 62.29, AUROC: 80.07 AUPR_IN: 80.93, AUPR_OUT: 76.60
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 69.30, AUROC: 85.91 AUPR_IN: 87.34, AUPR_OUT: 82.49
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [00:51<00:00,  3.41it/s]

Computing metrics on places365 dataset...
FPR@95: 52.47, AUROC: 82.91 AUPR_IN: 64.09, AUPR_OUT: 93.83
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 48.09, AUROC: 88.29 AUPR_IN: 76.62, AUPR_OUT: 93.00
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:04<00:00, 10.02it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    58.04  82.84    83.17     81.08 87.34
tin         53.79  84.73    86.34     81.29 87.34
nearood     55.92  83.78    84.76     81.18 87.34
mnist       30.30  92.64    74.58     98.79 87.34
svhn        40.29  91.69    80.48     96.90 87.34
texture     69.30  85.91    87.34     82.49 87.34
places365   52.47  82.91    64.09     93.83 87.34
farood      48.09  88.29    76.62     93.00 87.34
		FPR@95	AUROC	AUPR_IN	AUPR_OUT	ACC
msp	nearood	61.76	80.61	82.24	76.03	87.34
	farood	52.89	83.68	68.33	89.40	87.34
entropy	nearood	62.20	81.25	82.36	77.82	87.34
	farood	53.33	84.54	68.59	90.30	87.34
norm	nearood	56.71	84.46	85.31	82.87	87.34
	farood	43.10	89.82	77.98	94.02	87.34
cos	nearood	80.42	66.06	69.00	57.79	87.34
	farood	80.17	62.67	45.36	73.47	87.34
euc	nearood	80.42	66.06	69.00	57.79	87.34
	farood	80.17	62.67	45.36	73.47	87.34
neco	nearood	55.92	83.78	84.76	81.18	87.34
	farood	48.09	88.29	76.62	93.00	87.34

FPR@95	Best NearOOD AUROC	p